In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install torch torchvision timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 630.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 11.7 MB/s eta 0:00:00


In [ ]:
import torch

# Check if CUDA is available
is_cuda_available = torch.cuda.is_available()
print(f'CUDA Available: {is_cuda_available}')

# If CUDA is available, print CUDA version and device details
if is_cuda_available:
    print(f'CUDA Version: {torch.version.cuda}')
    print(f'Number of GPUs: {torch.cuda.device_count()}')
    for i in range(torch.cuda.device_count()):
        print(f'Device {i}: {torch.cuda.get_device_name(i)}')
else:
    print('CUDA is not available.')

CUDA Available: True
CUDA Version: 12.1
Number of GPUs: 1
Device 0: Tesla T4


In [ ]:
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
import cv2
import numpy as np
from torchmetrics.classification import MulticlassAccuracy, MulticlassPrecision, MulticlassRecall, MulticlassF1Score
from torchvision.models import vit_b_16

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Edge extraction function
def edge_extraction(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    return np.stack([edges] * 3, axis=-1)  # Stack edges to create a 3-channel image

# Custom dataset with noise in labels
class CariesDataset(Dataset):
    def __init__(self, img_folder, label_folder, transform=None, noise_level=0.1):
        self.img_folder = img_folder
        self.label_folder = label_folder
        self.image_paths = [f for f in os.listdir(img_folder) if f.endswith('.png')]
        self.transform = transform
        self.noise_level = noise_level

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_name = self.image_paths[idx]
        image_path = os.path.join(self.img_folder, img_name)
        label_path = os.path.join(self.label_folder, img_name)

        # Load image and apply edge extraction
        image = cv2.imread(image_path)
        image = edge_extraction(image)

        # Load label and convert to single binary value
        label_img = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)
        label = 1 if np.any(label_img > 0) else 0

        # Introduce label noise
        if random.random() < self.noise_level:
            label = 1 - label  # Flip the label

        if self.transform:
            image = self.transform(image)

        return image, label

# Data Augmentation and Transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),  # Resize for ViT input
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

# Load dataset
full_data = CariesDataset(img_folder='/content/drive/MyDrive/Xray/images_cut',
                          label_folder='/content/drive/MyDrive/Xray/labels_cut',
                          transform=transform)

# Split dataset
train_size = int(0.8 * len(full_data))
test_size = len(full_data) - train_size
train_data, test_data = random_split(full_data, [train_size, test_size])

# DataLoaders
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
test_loader = DataLoader(test_data, batch_size=16, shuffle=False)

# Load pre-trained Vision Transformer model
model = vit_b_16(pretrained=True)

# Replace the last layer of the model for binary classification
num_classes = 2
model.heads[-1] = nn.Linear(model.heads[-1].in_features, num_classes)  # Adjust output layer for binary classification
model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)

# Metrics
accuracy = MulticlassAccuracy(num_classes=2).to(device)
precision = MulticlassPrecision(num_classes=2).to(device)
recall = MulticlassRecall(num_classes=2).to(device)
f1_score = MulticlassF1Score(num_classes=2).to(device)

# Training loop with 100 epochs as specified
for epoch in range(100):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        running_loss += loss.item()

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch + 1}/100], Loss: {running_loss / len(train_loader):.4f}")
    #print(f"Accuracy on training set: {accuracy.compute():.4f}")

# Testing phase
model.eval()
with torch.no_grad():
    test_accuracy, test_precision, test_recall, test_f1 = 0, 0, 0, 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)

        # Calculate metrics
        test_accuracy += accuracy(preds, labels)
        test_precision += precision(preds, labels)
        test_recall += recall(preds, labels)
        test_f1 += f1_score(preds, labels)

    # Average metrics for the test set
    test_accuracy /= len(test_loader)
    test_precision /= len(test_loader)
    test_recall /= len(test_loader)
    test_f1 /= len(test_loader)

    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Test Precision: {test_precision:.4f}")
    print(f"Test Recall: {test_recall:.4f}")
    print(f"Test F1 Score: {test_f1:.4f}")

Using device: cuda


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:03<00:00, 101MB/s] 


Epoch [1/100], Loss: 0.4148
Epoch [2/100], Loss: 0.3579
Epoch [3/100], Loss: 0.3126
Epoch [4/100], Loss: 0.2481
Epoch [5/100], Loss: 0.2681
Epoch [6/100], Loss: 0.4238
Epoch [7/100], Loss: 0.2453
Epoch [8/100], Loss: 0.3150
Epoch [9/100], Loss: 0.4034
Epoch [10/100], Loss: 0.3853
Epoch [11/100], Loss: 0.3038
Epoch [12/100], Loss: 0.4705
Epoch [13/100], Loss: 0.2725
Epoch [14/100], Loss: 0.4652
Epoch [15/100], Loss: 0.3034
Epoch [16/100], Loss: 0.5076
Epoch [17/100], Loss: 0.2739
Epoch [18/100], Loss: 0.2975
Epoch [19/100], Loss: 0.2658
Epoch [20/100], Loss: 0.2020
Epoch [21/100], Loss: 0.2360
Epoch [22/100], Loss: 0.3142
Epoch [23/100], Loss: 0.3934
Epoch [24/100], Loss: 0.2841
Epoch [25/100], Loss: 0.2966
Epoch [26/100], Loss: 0.5602
Epoch [27/100], Loss: 0.2569
Epoch [28/100], Loss: 0.3847
Epoch [29/100], Loss: 0.3801
Epoch [30/100], Loss: 0.4902
Epoch [31/100], Loss: 0.2834
Epoch [32/100], Loss: 0.4642
Epoch [33/100], Loss: 0.3279
Epoch [34/100], Loss: 0.3536
Epoch [35/100], Loss: 0

In [ ]:
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
import cv2
import numpy as np
from torchmetrics.classification import MulticlassAccuracy, MulticlassPrecision, MulticlassRecall, MulticlassF1Score
from torchvision.models import vgg19  # Change here

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Edge extraction function
def edge_extraction(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    return np.stack([edges] * 3, axis=-1)  # Stack edges to create a 3-channel image

# Custom dataset with noise in labels
class CariesDataset(Dataset):
    def __init__(self, img_folder, label_folder, transform=None, noise_level=0.1):
        self.img_folder = img_folder
        self.label_folder = label_folder
        self.image_paths = [f for f in os.listdir(img_folder) if f.endswith('.png')]
        self.transform = transform
        self.noise_level = noise_level

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_name = self.image_paths[idx]
        image_path = os.path.join(self.img_folder, img_name)
        label_path = os.path.join(self.label_folder, img_name)

        # Load image and apply edge extraction
        image = cv2.imread(image_path)
        image = edge_extraction(image)

        # Load label and convert to single binary value
        label_img = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)
        label = 1 if np.any(label_img > 0) else 0

        # Introduce label noise
        if random.random() < self.noise_level:
            label = 1 - label  # Flip the label

        if self.transform:
            image = self.transform(image)

        return image, label

# Data Augmentation and Transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),  # Resize for VGG input
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

# Load dataset
full_data = CariesDataset(img_folder='/content/drive/MyDrive/Xray/images_cut',
                          label_folder='/content/drive/MyDrive/Xray/labels_cut',
                          transform=transform)

# Split dataset
train_size = int(0.8 * len(full_data))
test_size = len(full_data) - train_size
train_data, test_data = random_split(full_data, [train_size, test_size])

# DataLoaders
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
test_loader = DataLoader(test_data, batch_size=16, shuffle=False)

# Load pre-trained VGG19 model
model = vgg19(pretrained=True)  # Change here

# Replace the last layer of the model for binary classification
num_classes = 2
model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, num_classes)  # Adjust output layer for binary classification
model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)

# Metrics
accuracy = MulticlassAccuracy(num_classes=2).to(device)
precision = MulticlassPrecision(num_classes=2).to(device)
recall = MulticlassRecall(num_classes=2).to(device)
f1_score = MulticlassF1Score(num_classes=2).to(device)

# Training loop with 100 epochs as specified
for epoch in range(100):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        running_loss += loss.item()

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch + 1}/100], Loss: {running_loss / len(train_loader):.4f}")
    #print(f"Accuracy on training set: {accuracy.compute():.4f}")

# Testing phase
model.eval()
with torch.no_grad():
    test_accuracy, test_precision, test_recall, test_f1 = 0, 0, 0, 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)

        # Calculate metrics
        test_accuracy += accuracy(preds, labels)
        test_precision += precision(preds, labels)
        test_recall += recall(preds, labels)
        test_f1 += f1_score(preds, labels)

    # Average metrics for the test set
    test_accuracy /= len(test_loader)
    test_precision /= len(test_loader)
    test_recall /= len(test_loader)
    test_f1 /= len(test_loader)

    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Test Precision: {test_precision:.4f}")
    print(f"Test Recall: {test_recall:.4f}")
    print(f"Test F1 Score: {test_f1:.4f}")


Using device: cuda


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:06<00:00, 94.8MB/s]


Epoch [1/100], Loss: 0.4731
Epoch [2/100], Loss: 0.3281
Epoch [3/100], Loss: 0.3101
Epoch [4/100], Loss: 0.3008
Epoch [5/100], Loss: 0.4344
Epoch [6/100], Loss: 0.2481
Epoch [7/100], Loss: 0.3661
Epoch [8/100], Loss: 0.2401
Epoch [9/100], Loss: 0.4406
Epoch [10/100], Loss: 0.3403
Epoch [11/100], Loss: 0.2836
Epoch [12/100], Loss: 0.3492
Epoch [13/100], Loss: 0.3488
Epoch [14/100], Loss: 0.2795
Epoch [15/100], Loss: 0.3245
Epoch [16/100], Loss: 0.2779
Epoch [17/100], Loss: 0.3040
Epoch [18/100], Loss: 0.3393
Epoch [19/100], Loss: 0.2652
Epoch [20/100], Loss: 0.3622
Epoch [21/100], Loss: 0.2995
Epoch [22/100], Loss: 0.3069
Epoch [23/100], Loss: 0.2887
Epoch [24/100], Loss: 0.2417
Epoch [25/100], Loss: 0.4345
Epoch [26/100], Loss: 0.2763
Epoch [27/100], Loss: 0.2916
Epoch [28/100], Loss: 0.2369
Epoch [29/100], Loss: 0.4909
Epoch [30/100], Loss: 0.1764
Epoch [31/100], Loss: 0.3679
Epoch [32/100], Loss: 0.3349
Epoch [33/100], Loss: 0.2077
Epoch [34/100], Loss: 0.2448
Epoch [35/100], Loss: 0

In [ ]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.6/890.6 kB 30.5 MB/s eta 0:00:00
